# Entering up to 150 lineups
FanDuel provides a daily CSV that allows players to upload multiple lineups. We bring the CSV to this notebook and then match the names in our APi with the names as they're written in the CSV.

In [131]:
import pandas as pd
import numpy as np
from datetime import datetime
import os
import requests
import sqlite3
import re

In [132]:
def clean_name(name):
    # Remove periods between initials like C.J., D.J. (case-sensitive)
    #name = re.sub(r'\b([A-Z])\.\s*([A-Z])\.\b', r'\1\2', name)
    
    # Remove common suffixes like Jr., Sr., III, II, IV (case-sensitive)
    cleaned_name = re.sub(r'(\,|\.|Sr|Jr|III|II|IV)', '', name).strip()
    
    return cleaned_name

In [133]:
#Finds names as they're written in FanDuel and DraftKings that aren't matched in API
def get_names_not_in_api(site_list, api_list):
    api_names_set = set(api_list)
    site_names_set = set(site_list)
    names_not_in_api = site_names_set.difference(api_names_set)
    return names_not_in_api

This is handy! When we trained the model we put together a dictionary for each site with the names paired with their matches in the api. We'll continually update them and then reverse the keys and values.

In [134]:
fd_names_to_change = {'Nic Claxton': 'Nicolas Claxton', 'Moritz Wagner': 'Moe Wagner', 'Tristan Da Silva': 'Tristan da Silva',\
                     'Dominick Barlow': 'Dom Barlow', 'GG Jackson': 'Gregory Jackson', 'KJ Martin': 'Kenyon Martin',\
                      'Cam Thomas': 'Cameron Thomas', 'Ish Wainright': 'Ishmail Wainright', 'Jeenathan Williams': 'Nate Williams',\
                     'Tristan Da Silva': 'Tristan da Silva'}

In [135]:
dk_names_to_change = {'Nic Claxton': 'Nicolas Claxton', 'Moritz Wagner': 'Moe Wagner', 'Alexander Sarr': 'Alex Sarr',\
                     'Dominick Barlow': 'Dom Barlow', 'KJ Martin': 'Kenyon Martin', 'Cam Thomas': 'Cameron Thomas',\
                     'Ronald Holland': 'Ron Holland', 'Marjon Beauchamp': 'MarJon Beauchamp', 'David Jones Garcia': 'David Jones',\
                     'Jeenathan Williams': 'Nate Williams', 'Ish Wainright': 'Ishmail Wainright', 'Guillermo Hernangomez': 'Willy Hernangomez'}

In [136]:
name_changes_fd = {v: k for k, v in fd_names_to_change.items()}
name_changes_dk = {v: k for k, v in dk_names_to_change.items()}

In [137]:
name_changes_fd

{'Nicolas Claxton': 'Nic Claxton',
 'Moe Wagner': 'Moritz Wagner',
 'Tristan da Silva': 'Tristan Da Silva',
 'Dom Barlow': 'Dominick Barlow',
 'Gregory Jackson': 'GG Jackson',
 'Kenyon Martin': 'KJ Martin',
 'Cameron Thomas': 'Cam Thomas',
 'Ishmail Wainright': 'Ish Wainright',
 'Nate Williams': 'Jeenathan Williams'}

In [138]:
#This is the CSV downloaded from the site
site_df = pd.read_csv('FD_MultiCSV_022025.csv')

In [139]:
#We process the CSV in Google Sheets so that we have a column for each position
#and a column with each player's name as it's written on the site
site_df

,PG,PG.1,SG,SG.1,SF,SF.1,PF,PF.1,C,Nickname
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nikola Jokic
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Victor Wembanyama
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Giannis Antetokounmpo
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Luka Doncic
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Joel Embiid
...,...,...,...,...,...,...,...,...,...,...
311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DaRon Holmes
312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pacome Dadiet
313,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Justin Minaya
314,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Enrique Freeman


In [140]:
# url = "https://tank01-fantasy-stats.p.rapidapi.com/getNBAPlayerList"

# headers = {
# 	"x-rapidapi-key": "3103a75392msh7bce7c32fde122cp134393jsn4d42ed6d08a8",
# 	"x-rapidapi-host": "tank01-fantasy-stats.p.rapidapi.com"
# }

# response = requests.get(url, headers=headers)

# #pri#nt(response.json())

In [141]:
# result = response.json()

In [142]:
# len(result['body'])

In [143]:
# # Connect to SQLite database (creates a new one if it doesn't exist)
# conn = sqlite3.connect("nba_dfs_model.db")
# cursor = conn.cursor()

# # Create the players table
# cursor.execute('''
# CREATE TABLE IF NOT EXISTS api_players (
#     playerID TEXT PRIMARY KEY,
#     pos TEXT,
#     team TEXT,
#     longName TEXT,
#     teamID TEXT
# )
# ''')

# # Insert or update data
# for player in result['body']:
#     cursor.execute('''
#     INSERT OR REPLACE INTO api_players (playerID, pos, team, longName, teamID)
#     VALUES (?, ?, ?, ?, ?)
#     ''', (player['playerID'], player['pos'], player['team'], player['longName'], player['teamID']))

# # Commit and close connection
# conn.commit()
# conn.close()

# print("Database and api_players table created successfully!")

In [144]:
# Connect to the database
conn = sqlite3.connect("nba_dfs_model.db")
cursor = conn.cursor()

# Retrieve all rows from the players table
cursor.execute("SELECT longName FROM api_players")

# Fetch all distinct game_id values
api_names = cursor.fetchall()

# Convert the result into a list
api_names_list = [row[0] for row in api_names]

# Close the connection
conn.close()

In [145]:
len(api_names_list)

1040

In [146]:
#Creating a list of the site names on the downloaded CSV
fd_names_list = list(site_df['Nickname'])

In [147]:
#Applying the clean_name function to the site names and API names
fd_names_list = [clean_name(name) for name in fd_names_list]
api_names_list = [clean_name(name) for name in api_names_list]

In [148]:
#An one-off replacement
api_names_list = ["Trevor Hudgins" if name == "Trevor  Hudgins" else name for name in api_names_list]

In [149]:
#Changing the API names, the ones in the model, so that they match the names in the downloaded CSV from the site
api_names_list = [name_changes_fd.get(item, item) for item in api_names_list]

In [150]:
#Changing the lists to sets so that we can find any mismatches the fell through the cracks.
api_names_set = set(api_names_list)
fd_names_set = set(fd_names_list)

In [151]:
fd_not_in_api = set.difference(fd_names_set, api_names_set)

In [152]:
len(fd_not_in_api)

1

# Fuzzy matching
Using fuzzy matching to find a names in the API that could match the names as they're written in FanDuel

In [153]:
from rapidfuzz import process, fuzz

In [154]:
SIMILARITY_THRESHOLD = 70

In [155]:
#This function takes the unmatched site name and looks for matchins in the api
def fuzzy_match(name, dk_names):
    match, score, _ = process.extractOne(name, dk_names, scorer=fuzz.token_sort_ratio)
    return match if score >= SIMILARITY_THRESHOLD else None

In [156]:
fd_not_in_api = list(fd_not_in_api)

In [157]:
fd_not_in_api

['Tristan da Silva']

In [158]:
matches_in_api = [fuzzy_match(x, api_names_list) for x in fd_not_in_api]

In [159]:
matches_in_api

['Tristan Da Silva']

In [160]:
#I guess here we just update the appropriate dictionary and run the notebook again
names_to_change = dict(zip(matches_in_api, fd_not_in_api))

In [161]:
names_to_change

{'Tristan Da Silva': 'Tristan da Silva'}

In [ ]:
#So now that all the names match ...